# Getting Json Outputs from Llama3 using LangChain 

In [1]:
import json
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [2]:
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
    "name": {"title": "Name", "description": "The person's name", "type": "string"},
    "age": {"title": "Age", "description": "The person's age", "type": "integer"},
    "fav_food": {
    "title": "Fav Food",
    "description": "The person's favorite food",
    "type": "string",
    "required": ["name", "age"], }
    }
}

In [5]:
# keep the model loaded indefinitely temperature=0.1,
llm = ChatOllama(model="llama3", format="json", keep_alive=-1, max_new_tokens=512)

messages = [
HumanMessage (content="Please tell me about a person using the following JSON schema:"),
HumanMessage (content="{schema}"),
HumanMessage (content="Now, considering the schema, tell me about a person named Lucas who is 17 years old and loves pizza & pancakes."),
]

In [6]:
prompt = ChatPromptTemplate.from_messages(messages)

In [7]:
prompt

ChatPromptTemplate(input_variables=[], messages=[HumanMessage(content='Please tell me about a person using the following JSON schema:'), HumanMessage(content='{schema}'), HumanMessage(content='Now, considering the schema, tell me about a person named Lucas who is 17 years old and loves pizza & pancakes.')])

In [8]:
# converting json schema to a string
dumps = json.dumps(json_schema, indent=2)

In [9]:
chain = prompt | llm | JsonOutputParser()

In [10]:
response = chain.invoke({"schema": dumps})

In [11]:
print(response)
print(type(response))

{'name': 'Lucas', 'age': 17, 'interests': ['pizza', 'pancakes']}
<class 'dict'>


We can turn this structured output functionality into a function calling pipeline.

In [12]:
def some_tool_that_prints_info_from_dict(json_dict):
    for key in json_dict.keys():
        print(f"{key}: {json_dict[key]}")

some_tool_that_prints_info_from_dict(response)

name: Lucas
age: 17
interests: ['pizza', 'pancakes']
